Beatriz Passoni Nunes 


### Importação do banco de dados
Durante esse processo, foram necessárias algumas modificações para garantir a compatibilidade com as funcionalidades da aplicação.
A única modificação feita foi a exclusão da coluna "status" da tabela pedidos, que não era utilizada na aplicação, foi removida. 

A seguir, estão os comandos SQL que implementam essas adaptações no Oracle.

In [ ]:
-- Create sequences
CREATE SEQUENCE categoria_id_seq START WITH 1 INCREMENT BY 1;
CREATE SEQUENCE subcategoria_id_seq START WITH 1 INCREMENT BY 1;
CREATE SEQUENCE produto_num_seq START WITH 1 INCREMENT BY 1;
CREATE SEQUENCE pedido_produto_id_seq START WITH 1 INCREMENT BY 1;

-- Create tables
CREATE TABLE categoria (
    id_categoria INT DEFAULT categoria_id_seq.NEXTVAL,
    nome VARCHAR2(50) NOT NULL,
    CONSTRAINT pk_categoria PRIMARY KEY (id_categoria),
    CONSTRAINT uk_categoria_nome UNIQUE (nome)
);

CREATE TABLE subcategoria (
    id_subcategoria INT DEFAULT subcategoria_id_seq.NEXTVAL,
    nome VARCHAR2(50) NOT NULL,
    id_categoria INT,
    CONSTRAINT pk_subcategoria PRIMARY KEY (id_subcategoria),
    CONSTRAINT fk_categoria_id FOREIGN KEY (id_categoria)
        REFERENCES categoria (id_categoria) ON DELETE CASCADE
);

CREATE TABLE produto (
    num INT DEFAULT produto_num_seq.NEXTVAL,
    nome VARCHAR2(50) NOT NULL,
    preco NUMBER(10, 2) NOT NULL,
    descricao VARCHAR2(1000),
    estoque INT NOT NULL,
    id_subcategoria INT NOT NULL,
    CONSTRAINT pk_produto PRIMARY KEY (num),
    CONSTRAINT fk_subcategoria_id FOREIGN KEY (id_subcategoria)
        REFERENCES subcategoria (id_subcategoria) ON DELETE CASCADE
);

CREATE TABLE cliente (
    nome VARCHAR2(50) NOT NULL,
    sobrenome VARCHAR2(100) NOT NULL,
    sexo CHAR(1),
    data_nascimento DATE NOT NULL,
    cpf CHAR(14) NOT NULL,
    email VARCHAR2(80) NOT NULL,
    senha VARCHAR2(30) NOT NULL,
    CONSTRAINT pk_cliente PRIMARY KEY (cpf),
    CONSTRAINT uk_cliente_email UNIQUE (email),
    CONSTRAINT ck_cliente_sexo CHECK (sexo IN ('M', 'F')),
    CONSTRAINT ck_cliente_cpf CHECK (REGEXP_LIKE(cpf, '^\d{3}\.\d{3}\.\d{3}-\d{2}$'))
);

CREATE TABLE pedido (
    nota_fiscal CHAR(14) NOT NULL,
    cpf_cliente CHAR(14) NOT NULL,
    data_pedido DATE NOT NULL,
    hora_pedido TIMESTAMP NOT NULL,
    CONSTRAINT pk_pedido PRIMARY KEY (nota_fiscal),
    CONSTRAINT fk_cliente_id FOREIGN KEY (cpf_cliente)
        REFERENCES cliente (cpf) ON DELETE CASCADE
);

CREATE TABLE pedido_produto (
    id INT DEFAULT pedido_produto_id_seq.NEXTVAL,
    qnt_produto INT NOT NULL,
    nf_pedido CHAR(14) NOT NULL,
    num_produto INT NOT NULL,
    CONSTRAINT pk_pedido_produto PRIMARY KEY (id),
    CONSTRAINT fk_pedido_nf FOREIGN KEY (nf_pedido)
        REFERENCES pedido (nota_fiscal) ON DELETE CASCADE,
    CONSTRAINT fk_produto_num FOREIGN KEY (num_produto)
        REFERENCES produto (num) ON DELETE CASCADE,
    CONSTRAINT ck_qnt_produto CHECK (qnt_produto > 0)
);

### Carga das tabelas do banco de dados

In [ ]:
CREATE OR REPLACE PROCEDURE carga_dados(fator_escala IN NUMBER) AS
BEGIN
    -- Carregar categorias
    FOR i IN 1..(4 * fator_escala) LOOP
        INSERT INTO categoria (nome)
        VALUES ('Categoria ' || i);
    END LOOP;

    -- Carregar subcategorias
    FOR i IN 1..(10 * fator_escala) LOOP
        INSERT INTO subcategoria (nome, id_categoria)
        VALUES ('Subcategoria ' || i, MOD(i - 1, (4 * fator_escala)) + 1);
    END LOOP;

    -- Carregar produtos
    FOR i IN 1..(20 * fator_escala) LOOP
        INSERT INTO produto (nome, preco, descricao, estoque, id_subcategoria)
        VALUES (
            'Produto ' || i,
            ROUND(DBMS_RANDOM.VALUE(10, 500), 2), -- Preço aleatório entre 10 e 500
            'Descrição do Produto ' || i,
            TRUNC(DBMS_RANDOM.VALUE(1, 100)), -- Estoque aleatório entre 1 e 100
            MOD(i - 1, (10 * fator_escala)) + 1 -- Subcategoria aleatória
        );
    END LOOP;

    -- Carregar clientes
    FOR i IN 1..(5 * fator_escala) LOOP
        -- Gerar CPF diretamente no formato correto
        DECLARE
            cpf_formatado CHAR(14);
        BEGIN
            -- Gerar os primeiros 9 dígitos do CPF
            cpf_formatado := LPAD(TRUNC(DBMS_RANDOM.VALUE(100000000, 999999999)), 9, '0');
            -- Formatá-lo como XXX.XXX.XXX-XX
            cpf_formatado := SUBSTR(cpf_formatado, 1, 3) || '.' || 
                             SUBSTR(cpf_formatado, 4, 3) || '.' || 
                             SUBSTR(cpf_formatado, 7, 3) || '-' || 
                             LPAD(TRUNC(DBMS_RANDOM.VALUE(0, 99)), 2, '0');

            -- Inserir cliente com CPF formatado
            INSERT INTO cliente (nome, sobrenome, sexo, data_nascimento, cpf, email, senha)
            VALUES (
                'Cliente' || i,
                'Sobrenome' || i,
                CASE MOD(i, 2) WHEN 0 THEN 'M' ELSE 'F' END,
                TRUNC(TO_DATE('1930-01-01', 'YYYY-MM-DD') + DBMS_RANDOM.VALUE(0, 31046)), -- Data entre 1930 e 2015
                cpf_formatado,
                'cliente' || i || '@exemplo.com',
                'Senha' || i
            );
        END;
    END LOOP;

    -- Carregar pedidos
    FOR i IN 1..(10 * fator_escala) LOOP
        INSERT INTO pedido (nota_fiscal, cpf_cliente, data_pedido, hora_pedido)
        VALUES (
            LPAD(i, 14, '0'),
            (SELECT cpf FROM (SELECT cpf FROM cliente ORDER BY DBMS_RANDOM.VALUE) WHERE ROWNUM = 1), -- Cliente aleatório
            TRUNC(SYSDATE - DBMS_RANDOM.VALUE(1, 365)), -- Data aleatória no último ano
            SYSDATE - DBMS_RANDOM.VALUE(0, 1) -- Hora aleatória no último dia
        );
    END LOOP;

    -- Carregar itens de pedidos
    FOR i IN 1..(30 * fator_escala) LOOP
        INSERT INTO pedido_produto (qnt_produto, nf_pedido, num_produto)
        VALUES (
            TRUNC(DBMS_RANDOM.VALUE(1, 10)), -- Quantidade aleatória entre 1 e 10
            (SELECT nota_fiscal FROM (SELECT nota_fiscal FROM pedido ORDER BY DBMS_RANDOM.VALUE) WHERE ROWNUM = 1), -- Pedido aleatório
            (SELECT num FROM (SELECT num FROM produto ORDER BY DBMS_RANDOM.VALUE) WHERE ROWNUM = 1) -- Produto aleatório
        );
    END LOOP;

    COMMIT;
END;
/
SHOW ERRORS;

BEGIN
    carga_dados(2); -- Fator de escala 2
END;
/